In [1]:
import datetime
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import GridSearchCV

In [12]:
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
import datetime
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import GridSearchCV
import numpy as np
from scipy.stats import randint
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from sklearn.metrics import f1_score, make_scorer

f1 = make_scorer(f1_score , average='macro')

#  {'depth': 5, 'iterations': 500, 'learning_rate': 0.01}
print(" CATBOOST ".center(80, "="))

print("Loading dataset...")

df = pd.read_csv("/Users/pryda/Documents/Projets_Jedha/2_ConversioNRate/conversion_data_train.csv")

X = df.drop(["converted"], axis=1)
y = df["converted"]

# X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

categoricals = ["country", "new_user", "source"]

# model = CatBoostClassifier(iterations=500, learning_rate=0.01, depth=5, loss_function="Logloss", eval_metric="F1", cat_features=categoricals, verbose=100) # poop

# model = CatBoostClassifier(loss_function="Logloss", eval_metric="F1", cat_features=categoricals, verbose=100) # 0.777 train, 0.7517 test

# model = CatBoostClassifier(loss_function="Logloss", eval_metric="F1", cat_features=categoricals, verbose=100, depth=9, learning_rate=0.2, rsm=1) # f1-score on train set :  0.7910958904109588
# f1-score on test set :  0.7397585592717197

cbc = CatBoostClassifier(cat_features=categoricals, loss_function="Logloss")
param_dist = {
    # "learning_rate": np.linspace(0,0.2,5),
    # "depth": randint(3, 10),
    # "iterations": randint(100, 500),
    "rsm": np.linspace(0,1),
    "learning_rate": sp_randFloat(),
    "depth": sp_randInt(1, 10),
    "iterations": sp_randInt(10, 500),
}

model = RandomizedSearchCV(cbc, param_distributions=param_dist, n_iter=10, cv=10, scoring="f1", n_jobs=-1)

model.fit(X,y)

y_pred = model.predict(X)


# Print the tuned parameters and score
print(model.best_params_)
print(model.best_score_)

print("f1-score on train set : ", f1_score(y, y_pred))

# model.save_model("catboost_v1_randomsearch")

=================================== CATBOOST ===================================
Loading dataset...
0:	learn: 0.1179809	total: 815ms	remaining: 3m 7s
1:	learn: 0.0592487	total: 1.38s	remaining: 2m 38s
2:	learn: 0.0520193	total: 1.86s	remaining: 2m 21s
3:	learn: 0.0472382	total: 2.22s	remaining: 2m 5s
4:	learn: 0.0458141	total: 2.65s	remaining: 1m 59s
5:	learn: 0.0431074	total: 2.85s	remaining: 1m 46s
6:	learn: 0.0425628	total: 3.07s	remaining: 1m 38s
7:	learn: 0.0422626	total: 3.33s	remaining: 1m 32s
8:	learn: 0.0417733	total: 3.51s	remaining: 1m 26s
9:	learn: 0.0414157	total: 3.7s	remaining: 1m 21s
10:	learn: 0.0413891	total: 4.05s	remaining: 1m 21s
11:	learn: 0.0413034	total: 4.32s	remaining: 1m 18s
12:	learn: 0.0410390	total: 4.49s	remaining: 1m 15s
13:	learn: 0.0408612	total: 4.67s	remaining: 1m 12s
14:	learn: 0.0408604	total: 4.82s	remaining: 1m 9s
15:	learn: 0.0408405	total: 4.99s	remaining: 1m 7s
16:	learn: 0.0407993	total: 5.2s	remaining: 1m 5s
17:	learn: 0.0407868	total: 5.32s

In [7]:
df_test = pd.read_csv("/Users/pryda/Documents/Projets_Jedha/2_ConversioNRate/conversion_data_test.csv")

predictions = model.predict(df_test)
print(predictions)

pred_df = pd.DataFrame(columns=['converted'], data=predictions)
# pred_df = pd.DataFrame({ "converted": np.array(predictions)[:, -1] })
# pred_df.to_csv and insert datetime day hour minute in file name
pred_df.to_csv(f"conversion_data_test_predictions_LV_CBC_GS{datetime.datetime.now().strftime('%m%d_%H%M')}.csv", index=False)

# pred_df.to_csv(f"conversion_data_test_predictions_LV_catboost_split{datetime.datetime.now().timestamp()}.csv",
               #index=False)

[1 0 0 ... 0 1 0]
